# Debug the real spectra

So it doesn't work well with real data, so having taken out one example of real data, the sequence is

__NOTE__: These are ALL hybrids


1. `DLQTLALLL`
2. `DPQVAQLELGGEVEDPQVAQLELGGGPGAG`
3. `DLPVNSPMTKG`
4. `DLQTLALWSRM`
5. `DLQTLALNAAR`
6. `DLQTLALNAAR`
7. `DLQTLEVE`
8. `DLQTLALEVE`

so toy with the tool until we find out whats wrong

## Load params and run

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import runner
    

In [2]:
specPath = '/Users/zacharymcgrath/Desktop/nod2 data/hybridSpec/'
fastaPath = '/Users/zacharymcgrath/Desktop/nod2 data/filteredNOD2.fasta'
outputDir = '/Users/zacharymcgrath/Desktop/Experiment output/hybridReal/'
minPep = 3
maxPep = 30
tolerance = 20
relative_abundance_filter = 0.0
precursor_tolerance = 1
peak_filter = 30
verbose = True
scoringAlg = 'ion'

In [3]:
params = {
    'spectra_folder': specPath,
    'database_file': fastaPath,
    'output_dir': outputDir,
    'min_peptide_len': minPep,
    'max_peptide_len': maxPep,
    'tolerance': tolerance,
    'precursor_tolerance': precursor_tolerance,
    'peak_filter': peak_filter, 
    'relative_abundance_filter': relative_abundance_filter,
    'verbose': verbose, 
    'scoring_alg': scoringAlg, 
    'DEBUG': True
}

In [4]:
%%time
runner.run(params)

Loading database...
Done
Loading spectra...
Done
Making the protein mass set...
On protein 279/279 [100%]
Sorting the set of protein masses...
Done. Length of the list: 6385846
Time to build: 39.0007 seconds
Time to do merge: 2.2599 seconds
Creating an alignment for 8/8 [100%]
Finished search. Writting results to /Users/zacharymcgrath/Desktop/Experiment output/hybridReal/...
Could not make an alignment for 1/8 spectra (12%)
CPU times: user 47.2 s, sys: 1.17 s, total: 48.4 s
Wall time: 48.4 s


## Load and read results

In [5]:
import pandas as pd

outputFile = outputDir + 'summary.tsv'
hybridOutputFile = outputDir + 'hybrid_summary.tsv'

In [6]:
df = pd.read_csv(outputFile, sep='\t')
hdf = pd.read_csv(hybridOutputFile, sep='\t')

In [7]:
hdf.head(10)

,Unnamed: 0,left_proteins,right_proteins,sequence,hybrid_sequence,b_score,y_score,total_score,precursor_distance,entry name,id
0,0,['sp|P01325|INS1_MOUSE Insulin-1 OS=Mus muscul...,['sp|Q62186|SSRD_MOUSE Translocon-associated p...,DLQTLALLL,DLQT-LALLL,6,2,8,0.000194,0,NOD2_E3.21510.21510.2.pkl
1,1,['sp|P01325|INS1_MOUSE Insulin-1 OS=Mus muscul...,['sp|P26339|CMGA_MOUSE Chromogranin-A OS=Mus m...,DLQTLALWSRM,DLQTLAL-WSRM,5,8,13,0.000192,3,NOD2_E3.16373.16401.2.pkl
2,2,['sp|P63154|CRNL1_MOUSE Crooked neck-like prot...,['sp|Q02819|NUCB1_MOUSE Nucleobindin-1 OS=Mus ...,EVQITALNARA,EVQITA-LNARA,4,8,12,0.504749,5,NOD2_E3.10635.10674.3.pkl
3,3,['sp|Q3UVY5|PCX4_MOUSE Pecanex-like protein 4 ...,['sp|Q9WTX5|SKP1_MOUSE S-phase kinase-associat...,ELTNLEDL,ELTN-LEDL,6,2,8,0.000293,6,NOD2_E3.11427.11516.2.pkl


In [8]:
df.head(10)

,Unnamed: 0,proteins,sequence,b_score,y_score,total_score,precursor_distance,entry name,id
0,0,['sp|P17751|TPIS_MOUSE Triosephosphate isomera...,DLQRLEPGTM,1,3,4,0.497173,2,NOD2_E3.7065.7065.2.pkl
1,1,['sp|Q920A5|RISC_MOUSE Retinoid-inducible seri...,LDTQLKPRNT,3,4,7,0.000812,4,NOD2_E3.10614.10681.2.pkl
2,2,[],DLQTLALEVE,7,4,11,0.004477,7,NOD2_E3.16920.17393.2.pkl


In [9]:
import json

s = json.load(open('/Users/zacharymcgrath/Desktop/Experiment output/singleReal/summary.json', 'r'))

In [10]:
for k, v in s.items():
    for a in v['alignments']:
        print(f'{a["sequence"]}: {a["total_score"]}, {a["b_score"]}, {a["y_score"]}')


DLQTLALLL: 10, 6, 2
DLQTLALGGL: 9, 6, 1
DLQTLALEV: 9, 6, 0
LDTQLKPRNT: 4, 2, 3
